# Battle of the Neighborhoods code notebook.
## Juan Prieto-Pena

### 1. Table of Contents
1. Table of Contents  
2. [Introduction](#Introduction)  
3. [Business Understanding](#Business_Understanding)  
    3.1. [Background](#Background)  
    3.2. [Problem description](#Problem_description)  
    3.3. [Target audience](#Target_audience)  
    3.4. [Success criteria](#Success_criteria)  
4. [Data Understanding](#Data_Understanding)  
    4.1 [Geographical data](#Geographical_data)  
    4.2 [Population data](#Population_data)  
    4.3 [Income data](#Income_data)  
    4.4 [Real estate data](#Real_estate_data)  
    4.5 [Crime data](#Crime_data)  
    4.6 [Amenities data](#Amenities_data)  

<a id='Introduction'></a>
### 2. Introduction
This is the main document for the IBM Data Science Professional certificate capstone project. Here, all the code relative to the project will be written and explained. Accompanying this document and its datasets there is a pdf file with the final report and a Power Point presentation. There will be some addtional auxiliary files created during this project, such as maps. It will be stated their name and location throughout this project.

This project will allow the certificate candidates to demonstrate the knowledge learnt during the eight previous courses. The topic is free and the only constraint is to use Foursquare location data (by using their API) as one of the data sets employed for the project.

<a id='Business_Understanding'></a>
### 3. Business Understanding

<a id='Background'></a>
#### 3.1 Background

Madrid is not only Spain's capital, but also its principal business hub. It is located in almost the geographical center of the Iberian peninsula and Spain's infrastructure stems from the city in a radial network. This makes Madrid very attractive for business due to its benefitial fiscal aids from its local and regional government, the ease of travel from and to the city and its culture of entrepreneurship.

The city is a major center for banking, finance, retailing, trade, media, services and tourism not only in the country, but in the European southwest. Madrid is an alpha city according to the 2018 Globalization and World Cities Research Network [(GaWC)]. According to GaWC, "Alpha level cities are linked to major economic states and regions and into the world economy, and are classified into four sections, Alpha ++, Alpha +, Alpha, and Alpha − cities".

Due to this, the cost of living in Madrid is higher than in other spanish cities, but the socioeconomic differences between the different neighborhoods of the city can be used to find affordable places to live.

[(GaWC)]: https://www.lboro.ac.uk/gawc/world2018t.html

<a id='Problem_description'></a>
#### 3.2 Problem description

Recently, a business in the city has been expanding its operations, and needs to recruit talent from outside the city (both nationally and internationally). To help their prospective employees, they want to analyze the level of living throughout the city to better recomend their new workforce where to settle. The insights derived from this analysis will give a good understanding of the socioeconomic conditions of the different neighborhoods of the city and will allow to tailor real estate recomendations to all their prospective employees independently of their salaries. 

The aim of this is to increase worker satisfaction with the company and reduce the employee churn rate and retain talent. A good worker satisfaction would also raise the customer's opinion on the company, with a posible possitive effect on their sales.

The key indicators employed to analyze Madrid's neighborhoods will be:

* Population
* Average income
* Crime level
* Amenities in the neighborhood
* Real estate and rent prices (per square meter)

<a id='Target_audience'></a>
#### 3.3 Target audience

The objective is to study the socioeconomic levels of the city in order to provide housing and living expenses recomendations to new employees of the client company. The company's management also expects to understand the rationale behind the recommendations made.

The insights extracted from this analysis would also interest anyone interested in living in the city.

<a id='Success_Criteria'></a>
#### 3.4 Success Criteria

The project will be considered successful if a tiered list of Madrid's neighborhoods based on socioeconomic and business diversity in the neighborhood can be presented to the client to inform its prospective employees of their living choices in the city.

<a id='Data_Understanding'></a>
### 4. Data Understanding

Several datasets pertaining the city of Madrid (Spain) will be used. All of them will be described in the following section. In this section an exploratory analysis of the data will also be performed. To do that, first we need to import the libraries that will be used in this section.

In [1]:
import folium
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import branca

In [2]:
#Code to see all the rows and columns in a pandas dataframe:

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

<a id='Geographical_data'></a>
#### 4.1 Geographical data

All geographical data was taken from one source: The list of administrative divisions (current districts and neighborhoods) of Madrid, taken from the [open data page of the city of Madrid](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=46b55cde99be2410VgnVCM1000000b205a0aRCRD&vgnex). The files in geographic format for both neighborhoods and districts ( _Barrios en formato geográfico_ , and _Distritos en formato geográfico_ , respectively) contain not only the neighborhood and district names and codes for each of them, but also the polygon shapes (in .shp format) so they can be placed in a map. This files were converted to .geojson format by means of an [external converter](https://mygeodata.cloud/converter/shp-to-geojson). It is important to specify the output geographical coordinate reference system of the data to [WGS84](https://earth-info.nga.mil/GandG/publications/tr8350.2/tr8350_2.html), which is the [standard for geojson format](https://tools.ietf.org/html/rfc7946). Once this is done, the geojson files in '/data/geodata' are obtained, for both the neighborhoods and the districts.

The areas of the neighborhoods were taken from the csv files from "Relación de barrios (superficie y perímetro)" (List of neighborhoods, surface and perimeter) from the [same page](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=46b55cde99be2410VgnVCM1000000b205a0aRCRD&vgnex) as the geographic files.

The first thing that will be done is extracting the useful data (neighborhood name and number, district name and number, and geographical coordinates of the neighborhoods) from the geojson file.

In [3]:
#Path to the neighborhood .geojson file. Warning: if you are using this on your own you will need to change it.
path='D:\\Juan\\Documents\\Data Science\\IBM Data Science\\Capstone\\Coursera_capstone\\Battle of the Neighborhoods\\data\\geodata\\madrid_neighborhoods.geojson'

#We open the file with this
with open(path,"r") as f:
    data_neigh = json.load(f)

# #With this we can navigate the json file
# data.keys()
# data['features'][0].keys()
# data['features'][0]['properties']['NOMBRE']
# #We got this from the exploration of the json, and will be useful to construct the geographical data frame.
# dist='properties.NOMDIS'
# dist_num='properties.CODDIS'
# neig='properties.NOMBRE'
# neig_num='properties.CODBAR'
# coords = 'geometry.coordinates'

#Create the dataframe from the geojson
df_mad_1=gpd.GeoDataFrame.from_features(data_neigh, crs='EPSG:4326')

#We get rid of the columns that we dont need and rename the ones of interest
df_mad_1.drop(['OBJECTID', 'geodb_oid', 'CODBARRIO','CODDISTRIT', 'ORIG_FID'],axis=1,inplace=True)
df_mad_1.columns=['geometry','District code','District','Neighborhood code','Neighborhood']
df_mad_1.head()

,geometry,District code,District,Neighborhood code,Neighborhood
0,"POLYGON ((-3.68379 40.35021, -3.68379 40.34888...",17,Villaverde,172,San Cristobal
1,"POLYGON ((-3.65751 40.32893, -3.65991 40.32786...",17,Villaverde,173,Butarque
2,"POLYGON ((-3.69297 40.34585, -3.69339 40.34579...",17,Villaverde,175,Los Angeles
3,"POLYGON ((-3.68192 40.36130, -3.68152 40.36022...",17,Villaverde,174,Los Rosales
4,"POLYGON ((-3.70515 40.36368, -3.70606 40.35983...",17,Villaverde,171,"Villaverde Alto, Casco Histórico de Villaverde"


The centroids will be calculated with the 'Geometry' column of the geopandas dataframe.

In [4]:
#Calculation of centroids for the neighborhoods
cent_long=[]
cent_lat=[]
for i in df_mad_1['geometry']:
    cent_long.append(list(i.centroid.coords)[0][0])
    cent_lat.append(list(i.centroid.coords)[0][1])

We add the centroid latitude and longitude to the dataframe:

In [5]:
#We have all the information to create the geographic dataframe:
df_mad_1['Latitude']=cent_lat;
df_mad_1['Longitude']=cent_long;
df_mad_1['Neighborhood']=df_mad_1['Neighborhood'].apply(lambda x: x.title())
df_mad_1.sort_values(by=['Neighborhood code'], inplace=True)
df_mad_1.reset_index(drop=True,inplace=True)
df_mad_1.head()

,geometry,District code,District,Neighborhood code,Neighborhood,Latitude,Longitude
0,"POLYGON ((-3.70593 40.42029, -3.70634 40.42017...",01,Centro,011,Palacio,40.415417,-3.714071
1,"POLYGON ((-3.69194 40.40908, -3.69203 40.40870...",01,Centro,012,Embajadores,40.409239,-3.702463
2,"POLYGON ((-3.69805 40.41928, -3.69654 40.41874...",01,Centro,013,Cortes,40.414844,-3.696829
3,"POLYGON ((-3.69576 40.42764, -3.69512 40.42734...",01,Centro,014,Justicia,40.423661,-3.696677
4,"POLYGON ((-3.71186 40.43019, -3.71050 40.43006...",01,Centro,015,Universidad,40.425671,-3.707071


Now the data from the csv file with information about the neighborhood's surface will be extracted.

In [6]:
df_mad_2=pd.read_csv('data\\geodata\\CALLEJERO_VIGENTE_BARRIOS_201809.csv',sep=";",encoding='latin-1')
df_mad_2.columns=['Neighborhood code','District code','Neighborhood','Neighborhood with accents','Surface (m2)','Perimeter']
df_mad_2.drop(['Neighborhood code','District code','Neighborhood','Perimeter'],axis=1,inplace=True)
df_mad_2.columns=['Neighborhood','Surface (m2)']
df_mad_2['Neighborhood']=df_mad_2['Neighborhood'].apply(lambda x: x.title())
df_mad_2['Neighborhood']=df_mad_2['Neighborhood'].apply(lambda x: x.rstrip())
df_mad_2.head()

,Neighborhood,Surface (m2)
0,Palacio,1471085
1,Imperial,967500
2,Pacífico,750065
3,Recoletos,870857
4,El Viso,1708046


The names of the neighborhoods in this file and the geojson file are not the same in some cases. It is needed to clean the data a little bit so we can join the dataframes later. We will rename the names of the neighborhoods in both dataframes to ensure that the same naming criteria is followed in both cases. Also, there are three cases where there is no surface information:

In [7]:
df_mad_2[df_mad_2['Surface (m2)']==0]

,Neighborhood,Surface (m2)
59,Ensanche De Vallecas,0
60,Valderrivas,0
80,El Cañaveral,0


The missing information will be added manually with approximate data taken from Google Maps:

In [8]:
df_mad_2.loc[df_mad_2['Neighborhood']=='Ensanche De Vallecas','Surface (m2)']= 8070000
df_mad_2.loc[df_mad_2['Neighborhood']=='Valderrivas','Surface (m2)']= 646463.81
df_mad_2.loc[df_mad_2['Neighborhood']=='El Cañaveral','Surface (m2)']= 10540000

In [9]:
df_mad_2[df_mad_2['Surface (m2)']==0]

,Neighborhood,Surface (m2)


In [10]:
#Names to change in df_mad_1:
old_mad_1=['Fuentelareina',
'Puerta Del Angel',
'San Cristobal',
'Cármenes',
'Pilar',
'Jerónimos',
'Los Angeles',
'Apostol Santiago',
'Aguilas',
'Salvador']
    
new_mad_1=['Fuentelarreina',
'Puerta Del Ángel',
'San Cristóbal',
'Los Cármenes',
'El Pilar',
'Los Jerónimos',
'Los Ángeles',
'Apóstol Santiago',
'Las Águilas',
'El Salvador']

for i in range(len(old_mad_1)):
    df_mad_1.loc[df_mad_1.Neighborhood==old_mad_1[i],'Neighborhood']=new_mad_1[i]

#For df_mad_2

old_mad_2=['Villaverde Alto C.H.',
'Casco H.Vallecas',
'Casco H.Vicálvaro',
'Arguelles',
'Peña Grande',
'Casco H.Barajas',
'Zofio']

new_mad_2=['Villaverde Alto, Casco Histórico De Villaverde',
'Casco Histórico De Vallecas',
'Casco Histórico De Vicálvaro',
'Argüelles',
'Peñagrande',
'Casco Histórico De Barajas',
'Zofío']

for i in range(len(old_mad_2)):
    df_mad_2.loc[df_mad_2.Neighborhood==old_mad_2[i],'Neighborhood']=new_mad_2[i]

We have now all the information needed to create the geographical data frame.

This dataframe (df_mad_geo) contains, in the following order, the following information: 
1. Neighborhood code.  
2. Neighborhood name.
3. District name.
4. District code.
5. Latitude of the neighborhood centroid.
6. Longitude of the neighborhood centroid.
7. Surface of the neighborhood.

The dataframe will be indexed by neighborhood code order.

In [11]:
df_mad_geo = pd.merge(left=df_mad_1, right=df_mad_2, how='outer', left_on='Neighborhood', right_on='Neighborhood')
df_mad_geo.sort_values(by=['Neighborhood code'], inplace=True)
df_mad_geo.reset_index(drop=True,inplace=True)
df_mad_geo

,geometry,District code,District,Neighborhood code,Neighborhood,Latitude,Longitude,Surface (m2)
0,"POLYGON ((-3.70593 40.42029, -3.70634 40.42017...",01,Centro,011,Palacio,40.415417,-3.714071,1471085.0
1,"POLYGON ((-3.69194 40.40908, -3.69203 40.40870...",01,Centro,012,Embajadores,40.409239,-3.702463,1032822.0
2,"POLYGON ((-3.69805 40.41928, -3.69654 40.41874...",01,Centro,013,Cortes,40.414844,-3.696829,592070.0
3,"POLYGON ((-3.69576 40.42764, -3.69512 40.42734...",01,Centro,014,Justicia,40.423661,-3.696677,742034.0
4,"POLYGON ((-3.71186 40.43019, -3.71050 40.43006...",01,Centro,015,Universidad,40.425671,-3.707071,947641.0
...,...,...,...,...,...,...,...,...
126,"POLYGON ((-3.58252 40.46380, -3.58087 40.46228...",21,Barajas,211,Alameda De Osuna,40.456552,-3.591438,1961904.0
127,"POLYGON ((-3.55426 40.51134, -3.55513 40.50857...",21,Barajas,212,Aeropuerto,40.478558,-3.563446,25132277.0
128,"POLYGON ((-3.57464 40.47165, -3.57528 40.47147...",21,Barajas,213,Casco Histórico De Barajas,40.474005,-3.578866,609202.0
129,"POLYGON ((-3.58054 40.47959, -3.58055 40.47919...",21,Barajas,214,Timón,40.477568,-3.602764,9595252.0


The geojson file can also be used to show in the map the demarcation between the different neighborhoods (and districts) in the city. By default, the neighborhood layer is shown, but this can be changed in the map. The map will be saved as an html file for ease of visualization in github.

In [12]:
#Lets create the map
mad_lat = 40.486775;
mad_lon = -3.703790;

m=folium.Map(location=[mad_lat, mad_lon], zoom_start=11)
madrid_nei = "data\\geodata\\madrid_neighborhoods.geojson" # geojson file for the neighborhoods
madrid_dis = "data\\geodata\\madrid_districts.geojson" # geojson file for the districts

#Add a tooltip to all the neighborhoods in the geojson file
folium.GeoJson(madrid_nei,
               name='Neighborhoods of Madrid',
               tooltip=folium.features.GeoJsonTooltip(
                       fields=['NOMBRE','NOMDIS'],
                       aliases=['Neighborhood','District'],
                       labels=True,
                       sticky=True,
                       toLocaleString=True
                       ),
               ).add_to(m)

#Add a marker to the centroid of each neighborhood.
for i in range(0,len(df_mad_geo)):   
    folium.CircleMarker(location=[df_mad_geo.iloc[i]['Latitude'],df_mad_geo.iloc[i]['Longitude']],
                        radius=1,
                        color='black',
                        fill_color='black',
                        fill_opacity=0.3,
                       ).add_to(m)

folium.GeoJson(madrid_dis,
               name='Districts of Madrid',
               show=False,
               tooltip=folium.features.GeoJsonTooltip(fields=['NOMBRE'],
                       aliases=['District'],
                       labels=True,
                       sticky=True,
                       toLocaleString=True
                       ),
               ).add_to(m)

folium.LayerControl().add_to(m)

# #Jupyter only needs to call the variable of the map to render it (m), but in 
# #iPython we need to generate the map in a html like this:  
m.save("madrid_geo.html")

m

<a id='Population_data'></a>
#### 4.2 Population data   

Population data for the city of Madrid was taken from the data bank of the City Council of Madrid, [here](http://www-2.munimadrid.es/TSE6/control/seleccionDatosBarrio). The information extracted is the number of inhabitants per neighborhood.

In [13]:
df_mad_pop=pd.read_excel('data\\population\\202001_population.xls')
df_mad_pop

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Población por distrito y barrio
0,Fecha:,01-02-2020,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Total
3,Distrito,Barrio,Edad,Total
4,CENTRO,PALACIO,Total,23708
...,...,...,...,...
134,BARAJAS,CORRALEJOS,Total,7808
135,NaN,NaN,NaN,NaN
136,NaN,NaN,NaN,NaN
137,NaN,NaN,NaN,NaN


There are several cells with NaN values and other not useful text that will be cleaned. Also, the column names are not correct. We will clean that in the following cell. But there is one more important thing to note. All districts and neighborhoods are in upper case. We could try to normalize the names, but they are already ordered by district and neighborhood in the same way as the df_mad_geo dataframe, so we could

In [14]:
df_mad_pop.dropna(inplace=True)
df_mad_pop.columns=['District','Neighborhood','Count','Population']
df_mad_pop.drop([3],axis='index',inplace=True)
df_mad_pop.drop(['Count'],axis=1,inplace=True)
df_mad_pop.reset_index(drop=True,inplace=True)
df_mad_pop

,District,Neighborhood,Population
0,CENTRO,PALACIO,23708
1,CENTRO,EMBAJADORES,47151
2,CENTRO,CORTES,10760
3,CENTRO,JUSTICIA,18072
4,CENTRO,UNIVERSIDAD,33434
...,...,...,...
126,BARAJAS,ALAMEDA DE OSUNA,19873
127,BARAJAS,AEROPUERTO,1911
128,BARAJAS,CASCO H.BARAJAS,7735
129,BARAJAS,TIMON,12942


This dataset contains, in the following order:
1. District name.
2. Neighborhood name.
3. Population value with the most recent information from census data.

We can start aggregating all the data into one single dataset, called df_mad.

In [15]:
df_mad=pd.merge(left=df_mad_geo, right=df_mad_pop['Population'], how='outer', left_index=True, right_index=True)
df_mad

,geometry,District code,District,Neighborhood code,Neighborhood,Latitude,Longitude,Surface (m2),Population
0,"POLYGON ((-3.70593 40.42029, -3.70634 40.42017...",01,Centro,011,Palacio,40.415417,-3.714071,1471085.0,23708
1,"POLYGON ((-3.69194 40.40908, -3.69203 40.40870...",01,Centro,012,Embajadores,40.409239,-3.702463,1032822.0,47151
2,"POLYGON ((-3.69805 40.41928, -3.69654 40.41874...",01,Centro,013,Cortes,40.414844,-3.696829,592070.0,10760
3,"POLYGON ((-3.69576 40.42764, -3.69512 40.42734...",01,Centro,014,Justicia,40.423661,-3.696677,742034.0,18072
4,"POLYGON ((-3.71186 40.43019, -3.71050 40.43006...",01,Centro,015,Universidad,40.425671,-3.707071,947641.0,33434
...,...,...,...,...,...,...,...,...,...
126,"POLYGON ((-3.58252 40.46380, -3.58087 40.46228...",21,Barajas,211,Alameda De Osuna,40.456552,-3.591438,1961904.0,19873
127,"POLYGON ((-3.55426 40.51134, -3.55513 40.50857...",21,Barajas,212,Aeropuerto,40.478558,-3.563446,25132277.0,1911
128,"POLYGON ((-3.57464 40.47165, -3.57528 40.47147...",21,Barajas,213,Casco Histórico De Barajas,40.474005,-3.578866,609202.0,7735
129,"POLYGON ((-3.58054 40.47959, -3.58055 40.47919...",21,Barajas,214,Timón,40.477568,-3.602764,9595252.0,12942


In [16]:
colormap = branca.colormap.LinearColormap(
    vmin=df_mad['Population'].quantile(0.0), 
    vmax=df_mad['Population'].quantile(1), 
    colors=['red','orange','lightblue','green','darkgreen'],
    caption="Neighborhood population",
)

In [17]:
m_pop=folium.Map(location=[mad_lat, mad_lon], zoom_start=11)

tooltip = folium.features.GeoJsonTooltip(
    fields=['Neighborhood', 'District', 'Population'],
    aliases=['Neighborhood:', 'District:', 'Population:'],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

g = folium.GeoJson(
    df_mad,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["Population"])
        if x["properties"]["Population"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip
).add_to(m_pop)

colormap.add_to(m_pop)

m_pop.save("madrid_pop.html")

m_pop

<a id='Income_data'></a>
#### 4.3 Income data

The spanish Instituto Nacional de Estadística, INE, ([National Statistics Institute](https://www.ine.es/experimental/atlas/exp_atlas_tab.htm)) has all the data needed to obtain average household incomes in the city of Madrid, at a district level. It also contains information beyond the district level, but their sub-sections are different from that of neighborhoods and therefore will not be used.

<a id='Real_estate_data'></a>
#### 4.4 Real Estate data

The information for the average price of the square meter at a district level will be scraped from [Idealista data website](https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/madrid-comunidad/madrid-provincia/madrid/). Idealista is one of the biggest real estate agencies in spain, and they have a very interesting data analysis and visualization available to the public. Data from year 2019 for both rent and house sales will be taken for all districts of the city.

<a id='Crime_data'></a>
#### 4.5 Crime data

Crime data is not readily available from single sources. In order to have a general idea of the crime levels in the city, only [data from Madrid's Municipal Police](https://datos.madrid.es/sites/v/index.jsp?vgnextoid=bffff1d2a9fdb410VgnVCM2000000c205a0aRCRD&vgnextchannel=20d612b9ace9f310VgnVCM100000171f5a0aRCRD) will be taken. Excel files for each month of the year are available. The data corresponding to arrests will be taken for the year 2019 for all districts and a dataframe will be constructed containing the crime levels per district.

<a id='Amenities_data'></a>
#### 4.6 Amenities data

The number of venues in each neighborhood and district will be taken from Foursquare using their API. This data will be used to asess the level of access to amenities (restaurants, enteratainment...) in each neighborhood.

The data obtained will be combined for all neighborhoods (using information on a district level where neighborhood level is not available, such as real state, crime and income) and the data set will be fed to a K-means algorithm to segment the neighborhoods. The insights obtained will be discussed in the final report.